In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# *Main Code Based on CNN*

In [ ]:
###  Edition 1.1 


import torch
import torch.nn as nn

class ComplexConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(ComplexConv2d, self).__init__()
        self.real_conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.imag_conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
    
    def forward(self, x):
        real = x.real
        imag = x.imag
        
        real_out = self.real_conv(real) - self.imag_conv(imag)
        imag_out = self.real_conv(imag) + self.imag_conv(real)
        
        return torch.complex(real_out, imag_out)

    
class ComplexCNN(nn.Module):
    def __init__(self):
        super(ComplexCNN, self).__init__()
        self.conv1 = ComplexConv2d(5, 16, kernel_size=3, padding=1)
        self.conv2 = ComplexConv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = ComplexConv2d(32, 64, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(64 * 36 * 144, 1024)
        self.fc2_output1 = nn.Linear(1024, 144 * 2 * 5)
        self.fc2_output2 = nn.Linear(1024, 36 * 2 * 5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, kernel_size=2)
        
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, kernel_size=2)
        
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, kernel_size=2)
        
        x = x.view(x.size(0), -1)  # Flatten the tensor
        
        x = self.fc1(x)
        x = nn.functional.relu(x)
        
        output1 = self.fc2_output1(x)
        output1 = output1.view(-1, 144, 2, 5)
        
        output2 = self.fc2_output2(x)
        output2 = output2.view(-1, 36, 2, 5)
        
        return output1, output2


    
def train_model(model, train_loader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, outputs in train_loader:
            optimizer.zero_grad()
            outputs1, outputs2 = outputs
            
            outputs1_pred, outputs2_pred = model(inputs)
            
            loss1 = criterion(outputs1_pred, outputs1)
            loss2 = criterion(outputs2_pred, outputs2)
            
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')
    return model

In [ ]:
from scipy.io import loadmat
import os

In [ ]:
def load_data(data_folder):
    input_folder = os.path.join(data_folder, "Combined Dataset", "Input")
    output1_folder = os.path.join(data_folder, "Combined Dataset", "Output1")
    output2_folder = os.path.join(data_folder, "Combined Dataset", "Output2")
    
    input_files = [os.path.join(input_folder, file) for file in os.listdir(input_folder)]
    output1_files = [os.path.join(output1_folder, file) for file in os.listdir(output1_folder)]
    output2_files = [os.path.join(output2_folder, file) for file in os.listdir(output2_folder)]
    
    return input_files, output1_files, output2_files

In [ ]:
def load_mat_file(file_path, key):
    mat_data = loadmat(file_path)
    return torch.tensor(mat_data[key], dtype=torch.complex64)

In [ ]:
def process_data(input_files, output1_files, output2_files):
    input_real = []
    input_imag = []
    output1_real = []
    output1_imag = []
    output2_real = []
    output2_imag = []

    for i in range(len(input_files)):
        input_data = load_mat_file(input_files[i], 'ch')
        output1_data = load_mat_file(output1_files[i], 'Fopt_final')
        output2_data = load_mat_file(output2_files[i], 'Wopt_final')

        input_real.append(input_data.real)
        input_imag.append(input_data.imag)
        output1_real.append(output1_data.real)
        output1_imag.append(output1_data.imag)
        output2_real.append(output2_data.real)
        output2_imag.append(output2_data.imag)
        

    input_real = torch.stack(input_real)
    input_imag = torch.stack(input_imag)
    output1_real = torch.stack(output1_real)
    output1_imag = torch.stack(output1_imag)
    output2_real = torch.stack(output2_real)
    output2_imag = torch.stack(output2_imag)

    return input_real, input_imag, output1_real, output1_imag, output2_real, output2_imag

In [ ]:
def convert_to_complex(real,img):
    return torch.tensor(real+1j*img,dtype=torch.complex64) 


In [ ]:
input_files,output1_files, output2_files = load_data('/kaggle/input/dataset-final1')

In [ ]:
input_real, input_imag, output1_real, output1_imag, output2_real, output2_imag = process_data(input_files,output1_files,output2_files)

In [ ]:
print(input_real.shape)
print(output1_real.shape)
print(output2_real.shape)
print(input_imag.shape)
print(output1_imag.shape)
print(output2_imag.shape)

In [ ]:
inputs = convert_to_complex(input_real,input_imag)
outputs1 = convert_to_complex(output1_real,output1_imag)
outputs2 = convert_to_complex(output2_real,output2_imag)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Assuming data is loaded in variables inputs, outputs1, and outputs2
dataset = TensorDataset(inputs, outputs1, outputs2)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = ComplexCNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
model = train_model(model, train_loader, criterion, optimizer, num_epochs=25)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Define the complex tensors
inputs_complex = torch.tensor(input_real, dtype=torch.float32) + 1j * torch.tensor(input_imag, dtype=torch.float32)
outputs1_complex = torch.tensor(output1_real, dtype=torch.float32) + 1j * torch.tensor(output1_imag, dtype=torch.float32)

inputs = torch.tensor(inputs_complex, dtype=torch.complex64)
outputs1 = torch.tensor(outputs1_complex, dtype=torch.complex64)

# Assuming outputs2 is defined similarly
outputs2_complex = torch.tensor(output2_real, dtype=torch.float32) + 1j * torch.tensor(output2_imag, dtype=torch.float32)
outputs2 = torch.tensor(outputs2_complex, dtype=torch.complex64)

# Create the dataset and dataloader
dataset = TensorDataset(inputs, outputs1, outputs2)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the model (assuming ComplexCNN is defined)
model = ComplexCNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, outputs1, outputs2 in train_loader:
            optimizer.zero_grad()
            # Assuming the model returns two outputs
            outputs_pred1, outputs_pred2 = model(inputs)
            loss1 = criterion(outputs_pred1, outputs1)
            loss2 = criterion(outputs_pred2, outputs2)
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

    return model

# Train the model
model = train_model(model, train_loader, criterion, optimizer, num_epochs=25)


# Edition 1.2
*Error in codes*

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import scipy.io

# Define a custom dataset to handle the complex data
class ComplexDataset(Dataset):
    def __init__(self, inputs, outputs1, outputs2):
        self.inputs = inputs
        self.outputs1 = outputs1
        self.outputs2 = outputs2

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], (self.outputs1[idx], self.outputs2[idx])
'''
# Load .mat files
inputs_mat = scipy.io.loadmat('inputs.mat')['inputs']
outputs1_mat = scipy.io.loadmat('outputs1.mat')['outputs1']
outputs2_mat = scipy.io.loadmat('outputs2.mat')['outputs2']
'''
# Convert to PyTorch tensors
inputs = torch.tensor(inputs_mat, dtype=torch.complex64)
outputs1 = torch.tensor(outputs1_mat, dtype=torch.complex64)
outputs2 = torch.tensor(outputs2_mat, dtype=torch.complex64)


# Create dataset and dataloader
dataset = ComplexDataset(inputs, outputs1, outputs2)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the complex convolutional layer
class ComplexConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(ComplexConv2d, self).__init__()
        self.real_conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.imag_conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
    
    def forward(self, x):
        real = x.real
        imag = x.imag
        
        real_out = self.real_conv(real) - self.imag_conv(imag)
        imag_out = self.real_conv(imag) + self.imag_conv(real)
        
        return torch.complex(real_out, imag_out)

# Define the complex CNN model
class ComplexCNN(nn.Module):
    def __init__(self):
        super(ComplexCNN, self).__init__()
        self.conv1 = ComplexConv2d(5, 16, kernel_size=3, padding=1)
        self.conv2 = ComplexConv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = ComplexConv2d(32, 64, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(64 * 36 * 144, 1024)
        self.fc2_output1 = nn.Linear(1024, 144 * 2 * 5)
        self.fc2_output2 = nn.Linear(1024, 36 * 2 * 5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, kernel_size=2)
        
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, kernel_size=2)
        
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, kernel_size=2)
        
        x = x.view(x.size(0), -1)  # Flatten the tensor
        
        x = self.fc1(x)
        x = nn.functional.relu(x)
        
        output1 = self.fc2_output1(x)
        output1 = output1.view(-1, 144, 2, 5)
        
        output2 = self.fc2_output2(x)
        output2 = output2.view(-1, 36, 2, 5)
        
        return output1, output2

# Define the training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, outputs in train_loader:
            optimizer.zero_grad()
            outputs1, outputs2 = outputs
            
            outputs1_pred, outputs2_pred = model(inputs)
            
            loss1 = criterion(outputs1_pred, outputs1)
            loss2 = criterion(outputs2_pred, outputs2)
            
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')
    return model

# Initialize the model, criterion, and optimizer
model = ComplexCNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
model = train_model(model, train_loader, criterion, optimizer, num_epochs=25)
